# Manual Feature Engineering

Cuando tenemos una base de datos con muchas columnas, a veces puede interesarnos no ajustar los modelos con todas ellas.

Por tanto, ¿es necesario emplear todas las variables para el ajuste del modelo?

La respuesta es depende, depende del criterio que queramos fijarnos.

Existen varios criterios que podemos emplear:

* **Criterio estadístico del p-valor**: Empleando el método OLS del paquete de statsmodel, se puede valorar la probabilidad de que cierta variable sea significativa para el modelo.

* **Porcentaje de variabilidad explicada**: Y tener que ajustar un modelo PCA

* **En un modelo estilo Random Forest**: Listar la importancia de las variables en función de en cuántos árboles del bosque son relevantes.

En este notebook mostraré el primero de esos criterios, la aproximación más teórica.

In [2]:
# Importo las libs

from sklearn import linear_model
import statsmodels.api as sm
import pandas as pd

In [14]:
# Defino las cols

col_pred_0 = ["AirTime", "Distance", "TaxiIn", "TaxiOut", "DayOfWeek", "DayofMonth", "Month", "DepDelay", "WeatherDelay"]
col_target = "ArrDelay"

In [9]:
# Leemos dataset

data = pd.read_csv("../eggomPY_Datasets/AirlineDelays/DelayedFlights.csv")

In [10]:
# Limpiamos el dataset de NaN

df = data.dropna(subset=col_pred_0)
df = df.dropna(subset=["ArrDelay"])

In [11]:
# Sampleamos el dataset y lo capamos a un total de 5000 filas.

df = df.sample(frac=1,replace=True, random_state=12).head(5000)

In [12]:
# Reseteamos indices

df = df.reset_index()

In [15]:
# Defino un conjunto X con los datos de predicción y un conjunto con los datos objetivo

X = df[col_pred_0]
y = df[col_target]

In [16]:
# Creo un modelo de regresión lineal

lm = linear_model.LinearRegression()

# Lo fiteo usando todo el dataset

lm.fit(X,y)

# Predigo

y_guess = lm.predict(X)

In [20]:
# Al dataset X le añado un columna de 1s. Esto es para ver cuán de importante es una variable constante
X2 = sm.add_constant(X)
X2.head(3)

,const,AirTime,Distance,TaxiIn,TaxiOut,DayOfWeek,DayofMonth,Month,DepDelay,WeatherDelay
0,1.0,167.0,1065,8.0,59.0,5,16,5,143.0,0.0
1,1.0,45.0,224,7.0,9.0,6,12,1,118.0,0.0
2,1.0,38.0,236,5.0,32.0,2,5,2,180.0,0.0


In [22]:
# Creamos un modelo lineal usando 'Ordinary Least Squares'

ols = sm.OLS(y, X2)

# Lo fiteamos

estim_2 = ols.fit()

# Le pedimos un resumen de resultados (si usamos print nos lo devuelve tipo como el sofware R)
print(estim_2.summary())

                            OLS Regression Results                            
Dep. Variable:               ArrDelay   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                 1.974e+04
Date:                Mon, 04 May 2020   Prob (F-statistic):               0.00
Time:                        16:03:38   Log-Likelihood:                -18640.
No. Observations:                5000   AIC:                         3.730e+04
Df Residuals:                    4990   BIC:                         3.737e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -23.7301      0.586    -40.487   

Nos fijamos ahora en la columna 'P>|t|'. Estos son los p-valores.

Todas aquellas variables cuyo p-valor se encuentre por encima de 0.05 podemos inferir que tienen muy poca influencia en el modelo.

Para quitarlas tenemos que ir de 1 en 1.

En este caso quitaré "DayofMonth' que tiene un p-valor de 0.503.

Y repito la dinámica para obtener un nuevo OLS Summary:

In [23]:
col_pred_1 = ["AirTime", "Distance", "TaxiIn", "TaxiOut", "DayOfWeek", "Month", "DepDelay", "WeatherDelay"]

X = df[col_pred_1]
y = df[col_target]


X2 = sm.add_constant(X)

ols = sm.OLS(y, X2)
estim_2 = ols.fit()
print(estim_2.summary())

                            OLS Regression Results                            
Dep. Variable:               ArrDelay   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                 2.221e+04
Date:                Mon, 04 May 2020   Prob (F-statistic):               0.00
Time:                        16:08:13   Log-Likelihood:                -18640.
No. Observations:                5000   AIC:                         3.730e+04
Df Residuals:                    4991   BIC:                         3.736e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -23.5758      0.539    -43.743   

De nuevo, quitamos la variable con mayor p-valor: 'DayOfWeek'

In [26]:
col_pred_2 = ["AirTime", "Distance", "TaxiIn", "TaxiOut", "Month", "DepDelay", "WeatherDelay"]

X = df[col_pred_2]
y = df[col_target]


X2 = sm.add_constant(X)

ols = sm.OLS(y, X2)
estim_2 = ols.fit()
print(estim_2.summary())

                            OLS Regression Results                            
Dep. Variable:               ArrDelay   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                 2.537e+04
Date:                Mon, 04 May 2020   Prob (F-statistic):               0.00
Time:                        16:09:35   Log-Likelihood:                -18641.
No. Observations:                5000   AIC:                         3.730e+04
Df Residuals:                    4992   BIC:                         3.735e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -23.2298      0.465    -49.929   

Ahora quitamos la variable 'Month'

In [28]:
col_pred_3 = ["AirTime", "Distance", "TaxiIn", "TaxiOut", "DepDelay", "WeatherDelay"]

X = df[col_pred_3]
y = df[col_target]


X2 = sm.add_constant(X)

ols = sm.OLS(y, X2)
estim_2 = ols.fit()
print(estim_2.summary())

                            OLS Regression Results                            
Dep. Variable:               ArrDelay   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                 2.960e+04
Date:                Mon, 04 May 2020   Prob (F-statistic):               0.00
Time:                        16:10:23   Log-Likelihood:                -18642.
No. Observations:                5000   AIC:                         3.730e+04
Df Residuals:                    4993   BIC:                         3.734e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -23.6105      0.394    -59.967   

Quitaré también 'WeatherDelay'

In [30]:
col_pred_4 = ["AirTime", "Distance", "TaxiIn", "TaxiOut", "DepDelay"]

X = df[col_pred_4]
y = df[col_target]


X2 = sm.add_constant(X)

ols = sm.OLS(y, X2)
estim_2 = ols.fit()
print(estim_2.summary())

                            OLS Regression Results                            
Dep. Variable:               ArrDelay   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                 3.547e+04
Date:                Mon, 04 May 2020   Prob (F-statistic):               0.00
Time:                        16:11:10   Log-Likelihood:                -18646.
No. Observations:                5000   AIC:                         3.730e+04
Df Residuals:                    4994   BIC:                         3.734e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -23.6705      0.393    -60.190      0.0

Observando la tabla, parece un resultado ya aceptable.

Hemos eliminado las variables 'DayOfMonth', 'DayOfWeek' y 'Month'.

Sin embargo esta información parece, desde un punto de vista lógico, que debe tener influencia en los retrasos de las aerolíneas... ¿Porqué nuestro modelo las rehuye?

Muy sencillo:  Las variables que estaba usando ('DayofWeek', 'DayofMonth'., 'Month'...) son variables numéricas y las estaba tratando como números. Es por eso precisamente que no eran relevantes. Un uso correcto de estas variables hubiese sido categorizarlas como factores, es decir, si pertenecemos al día 1, al día 2 o al día 3. Lo que estaba haciendo Python era entender que día 2 era doble que día 1, día 3, triple que día 1... Es precisamente por eso por lo que salían variables no significativas aquí. ¿Cómo arreglarlo? Convirtiéndolas en "dummies". 

Para concluir, el modelo de regresión lineal es famoso por tener una explicación teórica muy bien estudiada. Aquí podemos obtener muchísima información de la importancia de la significación de la complejidad de los modelos.